In [1]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import pandas as pd
import pulp
import csv


pd.set_option('display.width', 1000)

In [36]:
# create dataframe of past salaries and past performance
fantasy_df = pd.read_csv("../Data/20200208__players.csv")

# Print the first five rows of data to the screen
fantasy_df.head()

,Name,Last,First,Player_Name,Starter,FD_Pts,FD_Sal,Pos,Team,H/A,Oppt,Team Score,Oppt Score,Minutes,Stat line,Date
0,"Holiday, Jrue",Holiday,Jrue,Jrue Holiday,1.0,58.2,6800.0,SG,nor,A,ind,124,117.0,31.78,31pt 6rb 10as 3st 4to 4trey 10-22fg 7-8ft,20200208
1,"Murray, Jamal",Murray,Jamal,Jamal Murray,1.0,50.5,6400.0,PG,den,A,pho,117,108.0,35.27,36pt 5rb 5as 1st 1bl 5to 6trey 14-17fg 2-3ft,20200208
2,"Murray, Dejounte",Murray,Dejounte,Dejounte Murray,1.0,50.3,4900.0,PG,sas,A,sac,102,122.0,27.9,17pt 9rb 9as 3st 2trey 6-11fg 3-4ft,20200208
3,"Hield, Buddy",Hield,Buddy,Buddy Hield,NaN,49.7,5400.0,SG,sac,H,sas,122,102.0,31.9,31pt 6rb 5as 2st 2to 9trey 11-17fg,20200208
4,"LeVert, Caris",LeVert,Caris,Caris LeVert,1.0,48.6,5300.0,SG,bkn,A,tor,118,119.0,33.22,37pt 3rb 4as 1st 1to 6trey 12-18fg 7-7ft,20200208


In [37]:
fantasy_df.columns

Index(['Name', 'Last', 'First', 'Player_Name', 'Starter', 'FD_Pts', 'FD_Sal', 'Pos', 'Team', 'H/A', 'Oppt', 'Team Score', 'Oppt Score', 'Minutes', 'Stat line', 'Date'], dtype='object')

In [38]:
fantasy_df = fantasy_df[['Player_Name', 'FD_Pts', 'FD_Sal', 'Pos']]
# fantasy_df['Player_Name'] = fantasy_df['Player_Name'].replace(' ', '_', regex=True)
fantasy_df

,Player_Name,FD_Pts,FD_Sal,Pos
0,Jrue Holiday,58.2,6800.0,SG
1,Jamal Murray,50.5,6400.0,PG
2,Dejounte Murray,50.3,4900.0,PG
3,Buddy Hield,49.7,5400.0,SG
4,Caris LeVert,48.6,5300.0,SG
...,...,...,...,...
291,Bol Bol,0.0,3500.0,C
292,DeMarcus Cousins,0.0,3500.0,C
293,Marc Gasol,0.0,4600.0,C
294,Zach Norvell Jr.,19.8,NaN,NaN


In [8]:
# fantasy_df.corr()['FDP']

In [39]:
fantasy_df = fantasy_df.dropna()
fantasy_df.head(296)

,Player_Name,FD_Pts,FD_Sal,Pos
0,Jrue Holiday,58.2,6800.0,SG
1,Jamal Murray,50.5,6400.0,PG
2,Dejounte Murray,50.3,4900.0,PG
3,Buddy Hield,49.7,5400.0,SG
4,Caris LeVert,48.6,5300.0,SG
...,...,...,...,...
289,Dwight Powell,0.0,3500.0,C
290,Tariq Owens,0.0,3500.0,C
291,Bol Bol,0.0,3500.0,C
292,DeMarcus Cousins,0.0,3500.0,C


In [41]:
fantasy_df['FD_Sal'] = fantasy_df['FD_Sal'].astype(int)

C:\Users\ddrag\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
fantasy_df

,Player_Name,FD_Pts,FD_Sal,Pos
0,Jrue Holiday,58.2,6800,SG
1,Jamal Murray,50.5,6400,PG
2,Dejounte Murray,50.3,4900,PG
3,Buddy Hield,49.7,5400,SG
4,Caris LeVert,48.6,5300,SG
...,...,...,...,...
289,Dwight Powell,0.0,3500,C
290,Tariq Owens,0.0,3500,C
291,Bol Bol,0.0,3500,C
292,DeMarcus Cousins,0.0,3500,C


In [9]:
# to_drop = ['TripleD', 'ADI', 'VMI', 'FG%', 'FT%']
# fantasy_df.drop(to_drop, axis=1, inplace = True)
# fantasy_df

In [43]:
fantasy_df = fantasy_df.rename(columns={'Player_Name':'Name'})

In [56]:
fantasy_df['Name'] = fantasy_df['Name'].replace(' ', '_', regex=True)
fantasy_df

,Name,FD_Pts,FD_Sal,Pos
0,Jrue_Holiday,58.2,6800,SG
1,Jamal_Murray,50.5,6400,PG
2,Dejounte_Murray,50.3,4900,PG
3,Buddy_Hield,49.7,5400,SG
4,Caris_LeVert,48.6,5300,SG
...,...,...,...,...
289,Dwight_Powell,0.0,3500,C
290,Tariq_Owens,0.0,3500,C
291,Bol_Bol,0.0,3500,C
292,DeMarcus_Cousins,0.0,3500,C


# LINEAR MODELING

In [57]:
# formulate linear programming problem
problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)

#Defining Variables

In [58]:
# initialize variables
playerPositions = fantasy_df['Pos'].tolist()
playerNames = fantasy_df['Name'].tolist()
playerCosts = tuple(fantasy_df['FD_Sal'])
playerPoints = tuple(fantasy_df['FD_Pts'])
salaryCap = 60000
numOfPlayers = range(len(playerNames))

In [59]:
# create variable to represent each player
playerInLineup = pulp.LpVariable.dicts("Player", [i for i in numOfPlayers], 0, 1, cat="Binary")


#Creating the Objective(s)

In [60]:
# Objective: maximize sum of player points in FanDuel
problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)
print(problem)

Optimal_Lineup:
MAXIMIZE
58.2*Player_0 + 50.5*Player_1 + 43.0*Player_10 + 40.0*Player_11 + 39.8*Player_12 + 39.1*Player_13 + 55.1*Player_130 + 55.0*Player_131 + 44.9*Player_132 + 44.1*Player_133 + 41.0*Player_134 + 39.2*Player_135 + 38.7*Player_136 + 38.4*Player_137 + 37.6*Player_138 + 37.4*Player_139 + 37.4*Player_14 + 35.8*Player_140 + 35.2*Player_141 + 32.1*Player_142 + 32.0*Player_143 + 31.9*Player_144 + 31.7*Player_145 + 31.2*Player_146 + 28.3*Player_147 + 28.2*Player_148 + 27.4*Player_149 + 36.9*Player_15 + 26.4*Player_150 + 25.3*Player_151 + 25.2*Player_152 + 24.8*Player_153 + 24.6*Player_154 + 24.6*Player_155 + 23.9*Player_156 + 23.9*Player_157 + 22.9*Player_158 + 22.9*Player_159 + 36.8*Player_16 + 22.8*Player_160 + 22.7*Player_161 + 22.4*Player_162 + 22.0*Player_163 + 21.4*Player_164 + 20.7*Player_165 + 20.5*Player_166 + 19.9*Player_167 + 17.9*Player_168 + 17.8*Player_169 + 35.8*Player_17 + 17.7*Player_170 + 17.3*Player_171 + 16.1*Player_172 + 15.8*Player_173 + 15.1*Player_174

# Defining the Contraints

In [61]:
# constraint n1: each player can only be chosen at most once
for i in numOfPlayers:
    problem += pulp.lpSum(playerInLineup[i]) <= 1
    

In [62]:
# constraint n2: sum of player costs cannot be more than or equal to FanDuel Salary cap ($ 60,000)
problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap

In [63]:
# constraint n3: teams can have up to 9 players
problem += pulp.lpSum(problem.variables()) == 9

#Player Positions

In [64]:
# constraint n4: number of PG = 2
pointguards = tuple(fantasy_df['Pos'] == 'PG')
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) <= 2

In [65]:
# constraint n5: number of SG = 2
shootinguard = tuple(fantasy_df['Pos'] == 'SG')
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) <= 2

In [66]:
# constraint n6: number of SF = 2
smallforward = tuple(fantasy_df['Pos'] == 'SF')
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) <= 2

In [67]:
# constraint n7: number of PF = 2
powerforward = tuple(fantasy_df['Pos'] == 'PF')
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) <= 2

In [68]:
# constraint n8: number of C = 1
center = tuple(fantasy_df['Pos'] == 'C')
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) <= 1

# Solving the Problem

In [69]:
# calculating predicted lineup
cost = 0
points = 0

predicted = set()
if problem.solve() == 1:
    for pos in range(len(numOfPlayers)):
        if playerInLineup[pos].value() == 1:
            name = playerNames[pos]
            predicted.add(name)
            cost += playerCosts[pos]
            points += playerPoints[pos]
            print (playerNames[pos], playerPositions[pos], playerCosts[pos],playerPoints[pos])
    print ('-------------------')
    print (f"The total cost is =: {cost}")
    print (f"The predicted FanDuel points: {points}")
else:
    print ('Error finding solution')

Jrue_Holiday SG 6800 58.2
Jamal_Murray PG 6400 50.5
Dejounte_Murray PG 4900 50.3
Caris_LeVert SG 5300 48.6
Anthony_Davis PF 10400 55.0
Andrew_Wiggins SF 7100 44.9
Harrison_Barnes SF 4700 38.4
Nemanja_Bjelica PF 5200 35.2
Deandre_Ayton C 9200 60.8
-------------------
The total cost is =: 60000
The predicted FanDuel points: 441.9
